**US Accident based on Infrastructure, State and Severity**

In [1]:
import os
import pandas as pd
import dask.array as da
import dask.dataframe as dd
from dask import compute
import numpy as np
from matplotlib import pyplot as plt
import dask
from dask.distributed import Client
from  dask.multiprocessing import get
from dask import compute,visualize
client=Client() # this set up 1 worker per core
client.cluster
import plotly.graph_objects as go
import plotly.express as px
import chart_studio.plotly as py
import cufflinks as cf
from plotly.offline import iplot, init_notebook_mode

%matplotlib inline
init_notebook_mode(connected=True)
cf.go_offline(connected=True)

cf.set_config_file(theme="pearl")

In [2]:
client=Client()
client.cluster

C:\Users\SOBHAN\anaconda3\lib\site-packages\distributed\node.py:151: UserWarning:

Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 58521 instead



In [3]:
dff= dd.read_csv('clean.csv')

In [21]:
df.head()

,ID,Severity,State,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop
0,A-1,3,OH,False,False,False,False,False,False,False,False,False,False,False,False,False
1,A-10,3,OH,False,False,False,False,False,False,False,False,False,False,False,False,False
2,A-100,2,OH,False,False,False,False,False,False,False,False,False,False,False,False,False
3,A-1000,2,CA,False,False,False,False,False,False,False,False,False,False,False,False,False
4,A-10000,3,CA,False,False,False,False,False,False,False,False,False,False,False,False,False


**Required columns for the Analysis**

In [4]:
df= dff[['ID','Severity', 'State', 'Amenity', 'Bump',
                        'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout',
                        'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']]

In [5]:
df.head()

,ID,Severity,State,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop
0,A-1,3,OH,False,False,False,False,False,False,False,False,False,False,False,False,False
1,A-10,3,OH,False,False,False,False,False,False,False,False,False,False,False,False,False
2,A-100,2,OH,False,False,False,False,False,False,False,False,False,False,False,False,False
3,A-1000,2,CA,False,False,False,False,False,False,False,False,False,False,False,False,False
4,A-10000,3,CA,False,False,False,False,False,False,False,False,False,False,False,False,False


In [6]:
df.compute().shape

(3402734, 16)

<font size="5">**Analysis of US accident on the basis of Infrastructure , State and Severity**</font>  

**Main Objective**
- Total count of accidents based on infrastructure(total) by state
- Total count of Accident based on infrastructure by Severity Level
- Total count of Accident by Infrastructure 
- Total count of Accidents on the basis of Severity by state (1 being least impacted and 4 being most impacted )
- Total count of most impactful accident on the basis of state
- Total count of Accident on basis of infrastructure(indivdual) by state 


1. Total count of  accidents based on infrastructure by state  

    Analyzing the difference in Total accidents and accidents related with the presence of infrastructure by State (*Number of Total Accident chart by state  is presented  above*)

In [7]:
Total_traffic= df.groupby('State')[[  'Amenity', 'Bump',
                        'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout',
                        'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']].sum()
Total_traffic= Total_traffic.sum(axis=1)


In [8]:
Total_traffic.compute().iplot(
                    kind="bar",
                    yTitle="no of accident",
                    xTitle="State",
                    linecolor="white",
                    title="Total count of  accidents based on infrastructure by state ", theme='pearl',
            
    )



<div style='background-color:navy; color:white; padding:20px'>
We can see huge difference in the number of accidents as whole by state and accidents due to presence of infrastructure nearby. California has nearly three times less  and North Dakota has 6 times less accidents compared to the total accidents as a whole. This may be of the reason of  infrastructure that helps to reduce accident. But looking at the number, more infrastructures could be planned in order to reduce the accident. 
</div>

2. Total count of Accident based on infrastructure by Severity Level

In [9]:
grp_sev = df.groupby('Severity').sum()
grp_sev.compute().iplot(kind = "bar", title="Total count of Accident based on infrastructure by Severity Level", yTitle="Values",
                            xTitle="Severity Level", opacity=0.6,linecolor="black")

 

<div style='background-color:navy; color:white; padding:20px'>
Looking at the figure, most of the accident are second level of severity(i.e. moderate).It is difficult to conclude whether the infrastructure is still less or it has not much effect on the moderate level of severity. For moderate to most  impactful severity, there's presence of high number of Traffic Signal and Junction nearby. Accidents nearby Traffic signal and juntions are often the result of not following the Traffic rules. There may be the need of more awareness on Traffic rules to reduce the accident along with its severity.
</div>

3. Total count of Accident by Infrastructure wise

In [10]:
grp_sev.sum(axis=0).compute().iplot(kind = "bar", title="Total count of Accident by Infrastructure wise", yTitle="Values",
                            xTitle="Infrastructure", opacity=0.6, linecolor = 'black')

<div style='background-color:navy; color:white; padding:20px'>
From the above figure, we can see that presence of traffic signal is the highest when the accident occured.Junction and Crossing are second and third highest. Often Traffic Signal and Crossing are regarded highly in reducing the accident but the graph shows otherwise. 
</div>

4. Total count of Accidents on the basis of Severity by state (1 being least impacted and 4 being most impacted )

In [11]:
df1 = df.compute().pivot_table(index=["State","Severity"], values="ID", aggfunc = "count").reset_index()

In [12]:
df1.Severity = df1.Severity.astype(str)

In [13]:

fig= px.bar(df1, y = 'ID', x = 'State', color = 'Severity',
             labels = {'ID': 'number of accidents',
                        'State': 'State'},
             title = 'Total count of Accidents on the basis of Severity by state',
            color_discrete_sequence = px.colors.qualitative.Pastel,
            category_orders = {'Severity' : [1,2,3,4]})
fig.show()

<div style='background-color:navy; color:white; padding:20px'>
California has the highest count of accidents in all severity level except 1 level of severity(minor). Arizona has the highest count of minor accidents. Texas and Florida comes second and third in terms of count of 2 and 3 level of severity. There are number of states which have very less accidents in all severity level. for e.g DC, Delaware, South Dakota e.t.c  
    
</div>

5. Total count of most impactful accident on the basis of state

In [14]:
most_impact_acc= df[df.Severity==4].groupby('State').State.count().compute()


In [15]:
most_impact_acc.iplot(kind='bar', title="Total count of most impactful accident on the basis of state", yTitle="Values",
                            xTitle="State", opacity=0.6, linecolor = 'black')

<div style='background-color:navy; color:white; padding:20px'>
Most Impactful accidents are often considered to be of grave injury to fatality. Florida leads tally in total count of most impactful accidents on the basis of state. It has 8636 cases of most severe accidents. And North Dakota has only 10 cases of such instance. California, Florida, Georgia, Michigan, Newyork, Ohio, Pennsylvenia and Virginia have more than 5000 cases of most impactful accidents whereas Alabama, Arkkansas, DC, Delaware, Kansas, Kentucky, Massachusetts, Minnesota, Mississippi, Nebraska and e.t.c have lower than 1000 cases.
</div>

6. Total count of Accident on basis of infrastructure nearby by state 

In [16]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [17]:
Inf_st_sev = df.groupby('State')[[ 'Amenity', 'Bump',
                        'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout',
                        'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']].sum()

In [18]:
chart = Inf_st_sev.compute().T

In [19]:
def infrastructure_by_state_severity(State):
    fig = chart[State].iplot(kind = 'bar',
                             layout = dict(
                             title = 'Total count of Accident on basis of infrastructure nearby by state  ' ))

In [20]:
interact(infrastructure_by_state_severity, State = chart.columns)

interactive(children=(Dropdown(description='State', options=('AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', '…

<function __main__.infrastructure_by_state_severity(State)>

<div style='background-color:navy; color:white; padding:20px'>
Presence of Traffic Signal nearby is highest for number of accidents in most of the states. The difference between the count of Traffic Signal and other attributes are quite large for many states. But California, Arkansas and Kansas being exception to the genral observation, it has count of junction much higher than other attributes. It may be due to the presence of high number of junction and less strict traffic rules.
</div>

**Summary**
We may have various conclusion on the basis of attributes used. From our attributes, the order of state change slightly if we take account of  
total accident by state and total accident by presence of infra-structure.  
By Total Count:  
1. California  
2. Texas  
3. Florida  
4. South Carolina   

By presence of infra-structure:  
1. California
2. Texas
3. Florida  
4. North-Carolina  

If we use other conditions such as by severity and each of the element of infrastructure, the order of states keeps changing. The only consistent observation that can be seen is the state with high population is highly likely to have high  number of accidents. In order to reduce accidents, more infrastructure  and more traffic awareness may be needed.



